In [8]:
import polars as pl
import pandas as pd
from dotenv import load_dotenv
from api.internal_api import SynthetixAPI, get_db_config

load_dotenv()

# constants
DATA_LOC = "~/repos/data/parquet-data"
db_config = get_db_config(streamlit=False)
api = SynthetixAPI(db_config, streamlit=False)


In [9]:
# get tlx data
df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/raw/eth_mainnet/getVaultCollateral/*.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_raw = df.to_pandas()


In [ ]:
df_raw


In [11]:
# get tlx data
df_pl = pl.scan_parquet(
    [
        f"{DATA_LOC}/clean/eth_mainnet/getVaultCollateral.parquet",
    ],
    include_file_paths="file_location",
)
df = df_pl.collect()
df_clean = df.to_pandas()


In [ ]:
df_clean

In [ ]:
df_clean[(df_clean["block_number"] > 21291400) & (df_clean["block_number"] < 21291500)]

In [ ]:
df_tlx["ts"] = df_tlx["blockTimestamp"]
df_tlx["market"] = (
    df_tlx["file_location"]
    .apply(lambda x: x.split("/")[-1].split("_")[0][:-1])
    .str.upper()
)


In [ ]:
# get prices
df_prices = api._run_query(
    """
SELECT distinct ts,
    block_number,
    market,
    last(price) over(partition by market order by ts) as price
FROM prod_optimism_mainnet.fct_v2_trades_optimism_mainnet
    """
)
print(df_prices)


In [ ]:
df_prices.head()

In [ ]:
df_tlx.head()

In [ ]:
# Convert 'ts' columns to integer timestamps in milliseconds
df_tlx["ts_int"] = df_tlx["ts"].astype("int64")
df_prices["ts_int"] = df_prices["ts"].astype("int64") // 10**6

# Sort DataFrames by 'ts_int' before merging
df_tlx_sorted = df_tlx.sort_values("ts_int")
df_prices_sorted = df_prices.sort_values("ts_int")

# Perform asof merge on 'ts_int'
df_merged = pd.merge_asof(
    df_tlx_sorted, df_prices_sorted, on="ts_int", by="market", direction="backward"
)

# Convert 'ts_int' back to datetime
df_merged["ts"] = pd.to_datetime(df_merged["ts_int"], unit="ms", utc=True)

# Drop the 'ts_int' column if no longer needed
df_merged = df_merged.drop(columns=["ts_int"])


In [ ]:
df_merged["volume"] = df_merged["leveragedTokenAmount"].apply(lambda x: int(x) / 1e18)

In [ ]:
df_merged.groupby("market")["volume"].sum().sort_values(ascending=False)

In [ ]:
# top accounts all time
top_accounts = (
    df_merged.groupby("account")["volume"].sum().sort_values(ascending=False).head(25)
)
top_accounts_30 = (
    df_merged[df_merged["ts"] > pd.Timestamp.now(tz="UTC") - pd.Timedelta(days=30)]
    .groupby("account")["volume"]
    .sum()
    .sort_values(ascending=False)
    .head(25)
)

In [ ]:
print(top_accounts)

In [ ]:
print(top_accounts_30)

In [ ]:
top_accounts.to_csv("top_accounts.csv")
top_accounts_30.to_csv("top_accounts_30.csv")